# Mongodb con python usando pymongo

## ¿Qué es mongodb?
Para comenzar podemos definir varios conceptos como el BIG Data siendo este definido como una gran colección de datos e información. Por otro lado tenemos que definir el concepto de NoSQL que se trata de una gran variedad de tecnologías usadas en bases de datos y que tienen diferentes caracteristicas como:

- No solo utilizan SQL como lenguaje de consultas.
- Ausencia de esquema
- No soportan JOIN
- ACID no garantizado (Atomicidad, Consistencia, Aislamiento, Durabilidad)
- Escalan horizontalmente

Como ejemplos de este tipo de bases de datos tenemos, redis para claves/valor, mongodb para documentos, neo4j para grafos y cassandra para columnas.

Podemos definir entonces mongodb como una base de datos que además de las características mencionadas anteriormente tiene las siguientes:

- Escalabilidad: No hay esquema y bloqueo a nivel de documento.
- Consistencia: Único servidor o varios para tener conseguir una alta disponibilidad.

Esta base de datos almacena sus documentos en formato JSon pero de forma binaria, es decir, formato Bson. El modelo de estos datos al ser a nivel de documento nos permite no tener columnas predefinidas o tipos de datos. En MongoDB cada documento tiene que contener un identificador que pueden ser de diferentes tipos, en caso de no ser definido por el usuario, MongoDB proporciona uno de tipo ObjectID como veremos más adelante.


## Trabajando con Python

### Configuración de entorno
#### Arranque del servicio
El primer paso será crear usuarios en el servidor de MongoDB y asignarles los roles correspondientes, para poder realizar accesos, consultas y modificaciones en las bases de datos y las colecciones.

Empezaremos arrancando el demonio de mongo con el init que tengamos por defecto. En nuestro caso es:

`systemctl start mongod`

Luego accederemos a través del cliente de MongoDB con

`mongo`


#### Creación de usuarios

Una vez nos encontremos en la consola de MongoDB, procederemos a create el usuario y su rol con el siguiente comando:

`db.createUser(
{ 
user: "usuario",
pwd:  "user123",
roles:
[
{ role:"readWrite",db:"db"}
] } );`

Donde user es el nombre del usuario, pwd es su contraseña, roles contiene los permisos del usuario(role) para cada base de datos(db). Si deseamos que el servidor verifique también las IP fuente del cliente o si se desea restringir las direcciones a las que puede conectarse dicho cliente, bastará con añadir los campos clientSource y serverAdress a la definición del usuario.

### Funciones básicas

Para comenzar a trabajar con la base de datos y python vamos a importar la libreria para usarlo.

In [79]:
import pymongo

A continuación vamos a establecer una conexión con la base de datos, para ello necesitamos importar MongoClient, conocer la dirección del host y el puerto, que por defecto es 27017. Se imprime por pantalla la conexión para saber que realmente funciona y estamos conectados

In [85]:
from pymongo import MongoClient

#MONGO_HOST = "10.42.0.1"
#MONG_HOST = "LOCALHOST"
#MONGO_PORT = 27017

#con = pymongo.MongoClient(MONGO_HOST, MONGO_PORT)

con = MongoClient()

print(con)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


Una vez conectados podemos crear una base de datos como se ve a continuación, pero esta base de datos no sera finalmente creada hasta que ¡¡no insertemos datos!!

In [104]:
mydb = con["ejemplo"]
print(con.list_database_names())

['admin', 'config', 'ejemplo', 'local']


Podemos mostrar el listado de bases de datos ya creadas y no aparecerá la base de datos "ejemplo" hasta que no insertemos contenido. Podemos obtener mas información de las distintas bases de datos creadas de la siguiente forma.

In [87]:
for db in con.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 40960, 'empty': False}
{'name': 'config', 'sizeOnDisk': 12288, 'empty': False}
{'name': 'local', 'sizeOnDisk': 40960, 'empty': False}


Para empezar a poder introducir contenido a la base de datos tenemos que tener creado una colección, podemos tener diferentes colecciones dentro de una misma base de datos, se realiza de la siguiente forma:

In [88]:
mycol = mydb["coleccion_ejemplo"]

Al igual que con las bases de datos también podemos listar las colecciones de una base de datos como se muestra a continuación, al no tener conenido en la base de datos que hemos creado, vamos a listar las colecciones de una que ya tenga contenido.

In [89]:
admincon = con["admin"]
print(admincon.list_collection_names())

['system.version']


Para poder meter información en la base de datos tenemos que tener la información en formato de diccionario, y como mencionamos antes podemos tener un identificador de nuestro tipo, por ejemplo un entero, o no ponerlo y que MongoDB lo gestione automaticamente con un ObjectID

In [90]:
#Informacion sin id
mydict1 = { "nombre": "Edu", "direccion": "Caceres" }

#Informacion con id
mydict2 = { "_id": 0, "nombre": "Pablo", "direccion": "Badajoz"}

mycol.insert_one(mydict1)
mycol.insert_one(mydict2)

Ahora vamos a insertar varios datos al mismo tiempo

In [91]:
mydict3 = {"_id": 1, "nombre": "Carlos", "apellidos": "Torres"}
mydict4 = {"_id": 2, "nombre": "Manuel", "estudiante": "Si"}

mycol.insert_many([mydict3, mydict4])

Una vez introducida la información que como se observa no tiene por que siempre llevar el mismo tipo de información, vamos a realizar algunas consultas básicas para ver que realmente hemos introducido correctamente la información en la base de datos. Podemos ver toda la información de una colección de la siguiente forma

In [92]:
results = mycol.find({})
for result in results:
    print (result)

{'_id': ObjectId('627945f3bab4f63e39c9d3a2'), 'nombre': 'Edu', 'direccion': 'Caceres'}
{'_id': 0, 'nombre': 'Pablo', 'direccion': 'Badajoz'}
{'_id': 1, 'nombre': 'Carlos', 'apellidos': 'Torres'}
{'_id': 2, 'nombre': 'Manuel', 'estudiante': 'Si'}


Podemos observar los diferentes tipos de id que tenemos como hemos mencionado anteriormente. Ahora vamos a buscar la información del id 1.

In [93]:
results = mycol.find({"_id": 1})
#Devuelve el cursor
print(results)
print()

#Devuelve el nombre asociado a id 1
for result in results:
    print(result["nombre"])


Carlos


Vamos a eliminar ahora información:

In [94]:
#Borramos
results = mycol.delete_one({"nombre": "Edu"})

#Comprobamos que este borrado
results = mycol.find({})
for result in results:
    print (result)

{'_id': 0, 'nombre': 'Pablo', 'direccion': 'Badajoz'}
{'_id': 1, 'nombre': 'Carlos', 'apellidos': 'Torres'}
{'_id': 2, 'nombre': 'Manuel', 'estudiante': 'Si'}


Ahora vamos a actualizar informacion, en nuestro caso vamos a cambiar el nombre usando el flag $set, pero podemos encontrar los diferentes flags en https://www.mongodb.com/docs/manual/reference/operator/update/

In [95]:
results = mycol.update_one({"_id": 1}, {"$set": {"nombre": "Jesus"}})

#Comprobamos que este actualizado
results = mycol.find({})
for result in results:
    print (result)

{'_id': 0, 'nombre': 'Pablo', 'direccion': 'Badajoz'}
{'_id': 1, 'nombre': 'Jesus', 'apellidos': 'Torres'}
{'_id': 2, 'nombre': 'Manuel', 'estudiante': 'Si'}


## Preparación de un data set para almacenar.

Ahora se va a proceder a preparar un dataset que contiene distinta información sobre coches para poder almacenarlos en la base de datos. Se va a comenzar importanto pandas para poder leer el dataset en formato csv, y se comprueba que se ha leido correctamente.

In [96]:
import pandas as pd

path = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(path, header=None)

print("Mostrando las 5 primeras filas del dataframe") 
df.head(5)

Mostrando las 5 primeras filas del dataframe


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


Ahora se va a crear una lista de columnas para que no aparezcan en formato numérico y podemos conocer que dato es cada columna.

In [97]:
headers = ["symboling","losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]
print("headers\n", headers)

df.columns = headers
df.head(10)

headers
 ['symboling', 'losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']


,symboling,losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
5,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
7,1,?,audi,gas,std,four,wagon,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,18920
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
9,0,?,audi,gas,turbo,two,hatchback,4wd,front,99.5,...,131,mpfi,3.13,3.40,7.0,160,5500,16,22,?


Ahora se va a proceder a eliminar las filas donde aparezcan datos que no son válidos.

In [98]:
#Eliminar filas sin datos de precio
df.dropna(subset=["price"], axis=0)

#Eliminar filas duplicadas
df.drop_duplicates()

#Eliminando filas con ? en losses
df = df[df.losses != '?']

df.head()


,symboling,losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
10,2,192,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101,5800,23,29,16430


En este punto de la ejecución si nos fijamos en el índice podremos observar que el índice tiene campos ausentes, que corresponde con las filas que se eliminaron en el paso anterior. Por lo que debemos reestablecer dicho índice para facilitar operaiones posteriores.

In [99]:
#Reestablecemos el índice del dataset, en el mismo objeto.
df.reset_index(drop = True, inplace=True)

#Volvemos a recuperar los datos una vez corregidos
df.head()

,symboling,losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
1,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
2,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
3,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
4,2,192,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101,5800,23,29,16430


### Tipado:

Algunas columnas poseen tipos que no corresponden con sus valores. Algunos ejemplos son los valores numéricos en forma de cadena de caracteres. En nuestro caso, las columnas con stroke, broke, normalized-losses, price y peak-rpm aparecen como objetos, siendo sus tipos correctos enteros y coma flotante.

In [100]:
df['bore'] = pd.to_numeric(df['bore'],errors='coerce')
df['stroke'] = pd.to_numeric(df['stroke'],errors='coerce')
df['losses'] = pd.to_numeric(df['losses'],errors='coerce')
df['price'] = pd.to_numeric(df['price'],errors='coerce')
df['peak-rpm'] = pd.to_numeric(df['peak-rpm'],errors='coerce')
df['num-of-cylinders'] = pd.to_numeric(df['num-of-cylinders'],errors='coerce')


#Comprobamos los tipos de las columnas del dataframe
df.dtypes

symboling              int64
losses                 int64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders     float64
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower            object
peak-rpm               int64
city-mpg               int64
highway-mpg            int64
price                  int64
dtype: object

## Transformando a diccionario
Para poder introducir todos los datos debemos convertir nuestros datos a formato diccionario, se realiza de la siguiente forma:

In [101]:
data = df.to_dict(orient = "records")

In [102]:
#Comprobamos que esta en formato diccionario.
data

[{'symboling': 2,
  'losses': 164,
  'make': 'audi',
  'fuel-type': 'gas',
  'aspiration': 'std',
  'num-of-doors': 'four',
  'body-style': 'sedan',
  'drive-wheels': 'fwd',
  'engine-location': 'front',
  'wheel-base': 99.8,
  'length': 176.6,
  'width': 66.2,
  'height': 54.3,
  'curb-weight': 2337,
  'engine-type': 'ohc',
  'num-of-cylinders': nan,
  'engine-size': 109,
  'fuel-system': 'mpfi',
  'bore': 3.19,
  'stroke': 3.4,
  'compression-ratio': 10.0,
  'horsepower': '102',
  'peak-rpm': 5500,
  'city-mpg': 24,
  'highway-mpg': 30,
  'price': 13950},
 {'symboling': 2,
  'losses': 164,
  'make': 'audi',
  'fuel-type': 'gas',
  'aspiration': 'std',
  'num-of-doors': 'four',
  'body-style': 'sedan',
  'drive-wheels': '4wd',
  'engine-location': 'front',
  'wheel-base': 99.4,
  'length': 176.6,
  'width': 66.4,
  'height': 54.3,
  'curb-weight': 2824,
  'engine-type': 'ohc',
  'num-of-cylinders': nan,
  'engine-size': 136,
  'fuel-system': 'mpfi',
  'bore': 3.19,
  'stroke': 3.4,
  

## Introduciendo el dataset en la base de datos
Vamos a introducir el dataset en la base de datos, para ello se van a seguir los pasos mencionados anteriormente, creando primero la base de datos, posteriormente creando la colección y finalmente introduciendo los datos.

In [103]:
mydb = con["car-information"]
print(con.list_database_names())

NameError: name 'client' is not defined

In [105]:
mycol = mydb["car-collection"]

In [118]:
for dato in data:
    mycol.insert_many([dato])

In [119]:
results = mycol.find({})
for result in results:
    print (result)

{'_id': ObjectId('627946f7bab4f63e39c9d3a3'), 'symboling': 2, 'losses': 164, 'make': 'audi', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'four', 'body-style': 'sedan', 'drive-wheels': 'fwd', 'engine-location': 'front', 'wheel-base': 99.8, 'length': 176.6, 'width': 66.2, 'height': 54.3, 'curb-weight': 2337, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 109, 'fuel-system': 'mpfi', 'bore': 3.19, 'stroke': 3.4, 'compression-ratio': 10.0, 'horsepower': '102', 'peak-rpm': 5500, 'city-mpg': 24, 'highway-mpg': 30, 'price': 13950}
{'_id': ObjectId('62794801bab4f63e39c9d3a4'), 'symboling': 2, 'losses': 164, 'make': 'audi', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'four', 'body-style': 'sedan', 'drive-wheels': '4wd', 'engine-location': 'front', 'wheel-base': 99.4, 'length': 176.6, 'width': 66.4, 'height': 54.3, 'curb-weight': 2824, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 136, 'fuel-system': 'mpfi', 'bore': 3.19, 'stroke': 3.4, 'co

## Realizando consultas
Ahora se van a realizar distintas consultas.

In [145]:
#Mostrando los vehiculos fabricados por audi
for resultado in mycol.find({},{"make": "audi"}):
  print(resultado)

{'_id': ObjectId('627946f7bab4f63e39c9d3a3'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3a4'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3a5'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3a6'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3a7'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3a8'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3a9'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3aa'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3ab'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3ac'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3ad'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3ae'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3af'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3b0'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3b1'), 'make': 'audi'}
{'_id': ObjectId('62794801bab4f63e39c9d3b2'), 'make': 'audi'}
{'_id': 

In [164]:

#Mostrando los vehiculos fabricados por audi y con 100 caballos.
for resultado in mycol.find({},{"make": "audi", "horsepower": "100"}):
  print(resultado)

{'_id': ObjectId('627946f7bab4f63e39c9d3a3'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3a4'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3a5'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3a6'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3a7'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3a8'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3a9'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3aa'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3ab'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3ac'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3ad'), 'make': 'audi', 'horsepower': '100'}
{'_id': ObjectId('62794801bab4f63e39c9d3ae'), 'make': 'audi', 'horsepower': '100'}
{'_i

Ahora vamos a crear una query para mostrar los vehiculos cuya marca de fabricación empiece por la letra S en adelante en orden alfabético.

In [160]:

myquery = { "make": { "$gt": "s" } }

resultados = mycol.find(myquery)

for resultado in resultados:
  print(resultado)

{'_id': ObjectId('62794802bab4f63e39c9d403'), 'symboling': 3, 'losses': 150, 'make': 'saab', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'two', 'body-style': 'hatchback', 'drive-wheels': 'fwd', 'engine-location': 'front', 'wheel-base': 99.1, 'length': 186.6, 'width': 66.5, 'height': 56.1, 'curb-weight': 2658, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 121, 'fuel-system': 'mpfi', 'bore': 3.54, 'stroke': 3.07, 'compression-ratio': 9.31, 'horsepower': '110', 'peak-rpm': 5250, 'city-mpg': 21, 'highway-mpg': 28, 'price': 11850}
{'_id': ObjectId('62794802bab4f63e39c9d404'), 'symboling': 2, 'losses': 104, 'make': 'saab', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'four', 'body-style': 'sedan', 'drive-wheels': 'fwd', 'engine-location': 'front', 'wheel-base': 99.1, 'length': 186.6, 'width': 66.5, 'height': 56.1, 'curb-weight': 2695, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 121, 'fuel-system': 'mpfi', 'bore': 3.54, 'stroke': 3.07

Ahora los que solamente empiecen por la letra C

In [165]:
myquery = { "make": { "$regex": "^s" } }

resultados = mycol.find(myquery)

for resultado in resultados:
  print(resultado)



{'_id': ObjectId('62794802bab4f63e39c9d403'), 'symboling': 3, 'losses': 150, 'make': 'saab', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'two', 'body-style': 'hatchback', 'drive-wheels': 'fwd', 'engine-location': 'front', 'wheel-base': 99.1, 'length': 186.6, 'width': 66.5, 'height': 56.1, 'curb-weight': 2658, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 121, 'fuel-system': 'mpfi', 'bore': 3.54, 'stroke': 3.07, 'compression-ratio': 9.31, 'horsepower': '110', 'peak-rpm': 5250, 'city-mpg': 21, 'highway-mpg': 28, 'price': 11850}
{'_id': ObjectId('62794802bab4f63e39c9d404'), 'symboling': 2, 'losses': 104, 'make': 'saab', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'four', 'body-style': 'sedan', 'drive-wheels': 'fwd', 'engine-location': 'front', 'wheel-base': 99.1, 'length': 186.6, 'width': 66.5, 'height': 56.1, 'curb-weight': 2695, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 121, 'fuel-system': 'mpfi', 'bore': 3.54, 'stroke': 3.07

Para mostrar resultados ordenados descententemente se puede realizar de la siguiente forma

In [183]:
resultados = mycol.find().sort("make", -1)

for resultado in resultados:
  print(resultado)

{'_id': ObjectId('62794802bab4f63e39c9d43c'), 'symboling': -2, 'losses': 103, 'make': 'volvo', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'four', 'body-style': 'sedan', 'drive-wheels': 'rwd', 'engine-location': 'front', 'wheel-base': 104.3, 'length': 188.8, 'width': 67.2, 'height': 56.2, 'curb-weight': 2912, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 141, 'fuel-system': 'mpfi', 'bore': 3.78, 'stroke': 3.15, 'compression-ratio': 9.5, 'horsepower': '114', 'peak-rpm': 5400, 'city-mpg': 23, 'highway-mpg': 28, 'price': 12940}
{'_id': ObjectId('62794802bab4f63e39c9d43d'), 'symboling': -1, 'losses': 74, 'make': 'volvo', 'fuel-type': 'gas', 'aspiration': 'std', 'num-of-doors': 'four', 'body-style': 'wagon', 'drive-wheels': 'rwd', 'engine-location': 'front', 'wheel-base': 104.3, 'length': 188.8, 'width': 67.2, 'height': 57.5, 'curb-weight': 3034, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 141, 'fuel-system': 'mpfi', 'bore': 3.78, 'stroke': 3.1

Ahora vamos a ver cuantos vehiculos con marca que empiecen por la letra S hay introducidos en la base de datos.

In [182]:
myquery = { "make": { "$regex": "^s" } }

resultados = mycol.count_documents(myquery)

print(resultados)

18


Con el operador and en las queries podemos hacer que se cumplan varias condiciones como se muestra a continuación

In [190]:
query = {
    "$and":
        [
            {
                "make": { "$regex": "^a" }
            },
            {
                "horsepower": "140"
            }
        ]
    }
resultados = mycol.find(query)

for resultado in resultados:
  print(resultado)

{'_id': ObjectId('62794801bab4f63e39c9d3a6'), 'symboling': 1, 'losses': 158, 'make': 'audi', 'fuel-type': 'gas', 'aspiration': 'turbo', 'num-of-doors': 'four', 'body-style': 'sedan', 'drive-wheels': 'fwd', 'engine-location': 'front', 'wheel-base': 105.8, 'length': 192.7, 'width': 71.4, 'height': 55.9, 'curb-weight': 3086, 'engine-type': 'ohc', 'num-of-cylinders': nan, 'engine-size': 131, 'fuel-system': 'mpfi', 'bore': 3.13, 'stroke': 3.4, 'compression-ratio': 8.3, 'horsepower': '140', 'peak-rpm': 5500, 'city-mpg': 17, 'highway-mpg': 20, 'price': 23875}


## Replicación en mongoDB

### ¿Qué es una réplica?
Las réplicas en mongoDB son un grupo de procesos mongod que mantienen el mismo data set. Cada réplica produce rebundancia y una alta disponibilidad, que hoy en día son fundamentales para el desarrollo de las actividades. En esta sección se va a explicar como se realiza una réplica en mongoDB, tendremos un servidor principal que es sobre el que se ha trabajado anteriormente y otro en forma de esclavo para la réplica.

### Configurando nuestro servidor principal
Partiendo de la base de datos sobre la que hemos trabajado, primeramente vamos a iniciar el servicio de mongod con los comandos:

`sudo systemctl start mongod`

`sudo systemctl daemon-reload`

Para garantizar que MongoDB se inicie tras un reinicio del sistema vamos a usar el siguiente comando:

`sudo systemctl enable mongod`

Por último vamos a verificar que MongoDB se ha iniciado correctamente:

`sudo systemctl status mongod`

Ahora vamos a configurar MondoDB para que este preparado para correr replica sets. Para ello vamos a editar el archivo de configuracion que se encuentra en `/etc/mongod.conf`

#network interfaces
net:
  port: 27017
  bindIp: 10.0.0.11 --> Lo rellenaremos con nuestra ip.
  
#replication:
replication:
  replSetName: "replicaset01" --> Lo rellenaremos con el nombre de nuestra réplica.
  
Por último vamos a reiniciar nuestro servicio con el comando:
`sudo systemctl restart mongod`

### Configurando nuestras réplicas

Una vez configurado el servidor principal vamos a crear nuestra réplica. Para comenzar partimos desde una máquina virtual configurada su red como puente y con el sistema operativo Debian. Después de realizar la instalación de MongoDB vamos a iniciar el proceso como lo hemos realizado en el apartado anterior y vamos a editar el archivo de configuración igual que se ha realizado antes, pero cambiando la ip por la de la máquina virtual. Por último también reiniciaremos el proceso.

### Añadiendo la réplica
Ahora vamos a mostrarle al servidor principal donde está nuestra réplica, para ello vamos a iniciar mongo en una terminal y realizar los siguientes pasos:

1. Iniciar la replicación con `rs.initiate()` la consola devolvera un ok = 1.
2. Pulsaremos enter para volver a la terminal primera de mongoDB
3. Añadimos los nodos donde halla réplica con `rs.add("ip de la réplica:puerto de la réplica")`

Para estar seguros que esta funcionando podemos comprobarlo desde el nodo principal mentiante el domando `rs.status()`
